In [1]:
import os
os.chdir("/home/sagemaker-user/data-bbog-integration-fabrica-personas/src")
import data_bbog_integration_fabrica_personas.pipelines.backtesting.nodes as backtesting
import pandas as pd
import polars as pl
import numpy as np
import logging
import matplotlib.pyplot as plt

## modelo 360

archivo_nombre = 'base_calificada_nueva_202503_v2'
producto = 'seg_desempleo' #seg_desempleo, seg_personales
rta = f's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica-personas/SEG/08-model_output/{producto}/{archivo_nombre}.parquet'
df = backtesting.load_parquet(rta)
df.to_csv(f'/home/sagemaker-user/data-bbog-integration-fabrica-personas/notebooks/Calibracion de Productos/{archivo_nombre}_{producto}.csv')
df

In [2]:
guardar_bigquery = True

In [3]:
archivo_nombre = 'campana_full_202508'
rta = f's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica-personas/modelo_360/{archivo_nombre}.parquet'
df = backtesting.load_parquet(rta)
display(df.head(2))
if guardar_bigquery:
    print('Guardando en bigquery')
    from google.oauth2 import service_account
    from pandas_gbq import to_gbq
    
    # # Define las credenciales (opcional si ya los tienes configuradas en SageMaker)
    # credentials = service_account.Credentials.from_service_account_file("ruta/a/tu/credencial.json")
    
    # Configura los parámetros de BigQuery
    proyecto_bq = "augusta-bbog-pro-activo"
    dataset_bq = "transit_output"
    tabla_bq = "Profundizacion_360_PN"
    
    # Guarda el DataFrame en BigQuery
    # to_gbq(df, f"{dataset_bq}.{tabla_bq}", project_id=proyecto_bq, credentials=credentials, if_exists="replace")
    # to_gbq(pyme_360, f"{dataset_bq}.{tabla_bq}", project_id=proyecto_bq, if_exists="replace")
    
    to_gbq(df, f"{dataset_bq}.{tabla_bq}", project_id=proyecto_bq, if_exists="append")
    
    print("✅ DataFrame guardado en BigQuery con éxito.")

else:
    df.to_csv(f'/home/sagemaker-user/data-bbog-integration-fabrica-personas/notebooks/Calibracion de Productos/{archivo_nombre}.csv')
    print('Ok')
df.shape

,hashvalue1,DEFINIT,probabilidad_1,probabilidad_2,probabilidad_3,probabilidad_4,probabilidad_5,probabilidad_6,probabilidad_7,probabilidad_8,...,decil_5,decil_6,decil_7,decil_8,decil_9,decil_10,propension_360,region,decil_ingreso,categoria
0,CC1083464350,C1083464350,0.020026,0.102845,0.003461,0.158488,0.270004,0.002280,0.025104,0.127205,...,8.0,8.0,8.0,5.0,6.0,2.0,10,Region_Andina,6.0,2.0
1,CC1083464371,C1083464371,0.158559,0.103560,0.045405,0.270015,0.025129,0.018352,0.003271,0.002003,...,8.0,6.0,7.0,7.0,3.0,3.0,10,Region_Caribe,7.0,2.0


Guardando en bigquery


## modelo Individual Producto

In [2]:
archivo_nombre = 'base_calificada_nueva_202503'
productos = ["FID"]
for producto in productos:
    rta = f's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica-personas/{producto}/08-model_output/{archivo_nombre}.parquet'
    df = backtesting.load_parquet(rta)
    display(df.head(2))
    df.to_csv(f'/home/sagemaker-user/data-bbog-integration-fabrica-personas/notebooks/Calibracion de Productos/{archivo_nombre}_{producto}.csv')
    print('OK')

,periodo,hashvalue1,y_pred,y_pred_proba,region,decil_ingreso,categoria
0,202503,CC1013660619,1,0.02116,Region_Central,10.0,3.0
1,202503,CC1193251945,1,0.02116,Region_Pacifica,10.0,3.0


OK


## Cruces con ADL:

In [9]:
producto = 'FID'
fecha_ejecucion = '202503'
ruta_adl = f's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica/{producto}/08-model_output/base_calificada_{fecha_ejecucion}.parquet'
ruta_adl

's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica/FID/08-model_output/base_calificada_202503.parquet'

In [11]:
archivo_nombre = 'base_calificada_nueva_202503'
rta = f's3://data-bbog-dev-sandbox-entity/casos-uso/fabrica-personas/{producto}/08-model_output/{archivo_nombre}.parquet'

In [23]:
## ajuste de ordenamiento de id
ids = 'hashvalue1'

In [26]:
df_aa = backtesting.load_parquet(rta)
df_adl = backtesting.load_parquet(ruta_adl)
df_aa = df_aa.set_index(ids)
df_adl = df_adl.set_index(ids)
df_adl = df_adl.loc[df_aa.index]

In [27]:
print(df_aa.shape)
print(df_adl.shape)

(4246937, 6)
(4246937, 6)


In [32]:
(df_aa['region'] == df_adl['region']).drop_duplicates().values

array([ True])

In [38]:
validation = (df_aa.drop('region',axis = 1).astype(float) - df_adl.drop('region',axis = 1).astype(float))
validation = validation.round(10)
validation = validation.replace(0, np.nan).dropna(how='all')
validation

,periodo,y_pred,y_pred_proba,decil_ingreso,categoria
hashvalue1,,,,,
